##Installing Apache Beam



In [1]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
!kaggle datasets download raidevesh05/movie-ratings-dataset

Dataset URL: https://www.kaggle.com/datasets/raidevesh05/movie-ratings-dataset
License(s): ODbL-1.0
  0% 0.00/20.2k [00:00<?, ?B/s]
100% 20.2k/20.2k [00:00<00:00, 29.7MB/s]


In [4]:
!unzip movie-ratings-dataset.zip

Archive:  movie-ratings-dataset.zip
  inflating: movie_ratings.csv       


##Creating Apache Beam Pipeline

In [5]:
class ExtractAndCountWords(beam.PTransform):
    def expand(self, pcoll):
        return (
            pcoll
            | 'Extract Words' >> beam.FlatMap(lambda x: x.split())
            | 'Count Words' >> beam.combiners.Count.PerElement()
        )


##Pipeline I/O

In [6]:
with beam.Pipeline() as pipeline:
    (
        pipeline
        | 'Read File' >> beam.io.ReadFromText('movie_ratings.csv')
        | 'Transform Data' >> beam.Map(lambda x: x.upper())
        | 'Write to File' >> beam.io.WriteToText('output.txt')
    )


##Triggers and Windowing

In [7]:
from apache_beam.transforms.window import FixedWindows

with beam.Pipeline() as pipeline:
    (
        pipeline
        | 'Create Data' >> beam.Create([('a', 1), ('b', 2), ('a', 3)])
        | 'Window' >> beam.WindowInto(FixedWindows(10))
        | 'Sum per Key' >> beam.CombinePerKey(sum)
        | 'Print Results' >> beam.Map(print)
    )

('a', 4)
('b', 2)


##ParDo

In [8]:
class CustomTransform(beam.DoFn):
    def process(self, element):
        yield element.upper()

with beam.Pipeline() as pipeline:
    (
        pipeline
        | 'Read' >> beam.Create(['hello', 'world'])
        | 'Transform' >> beam.ParDo(CustomTransform())
        | 'Write' >> beam.io.WriteToText('output.txt')
    )

##Streaming

In [9]:
def generate_data():
    for i in range(10):
        yield f"Data {i}"

with beam.Pipeline() as pipeline:
    (
        pipeline
        | 'Read Stream' >> beam.Create(generate_data())
        | 'Print Stream' >> beam.Map(print)
    )

Data 0
Data 1
Data 2
Data 3
Data 4
Data 5
Data 6
Data 7
Data 8
Data 9
